# 🧪 Teste Interativo dos Nós - WhatsApp Bot

Este notebook permite testar cada nó individualmente, similar ao fluxo do n8n.

**Vantagens:**
- ✅ Executa um nó por vez
- ✅ Visualiza input e output de cada nó
- ✅ Pode modificar o estado entre nós
- ✅ Debugging visual em tempo real

---

## 📦 CÉLULA 1: Imports e Setup

In [ ]:
import sys
import os

# Adicionar diretório raiz ao path
sys.path.insert(0, os.path.abspath('..'))

# Imports dos nós (comentados até instalar dependências)
# from src.models.state import AgentState, criar_estado_inicial
# from src.nodes.webhook import validar_webhook, verificar_cliente, cadastrar_cliente
# from src.nodes.media import processar_texto, processar_audio, processar_imagem
# from src.nodes.queue import gerenciar_fila, aguardar_mensagens
# from src.nodes.agent import processar_agente
# from src.nodes.response import fragmentar_resposta, enviar_respostas

import asyncio
import json
from pprint import pprint
from datetime import datetime

print("✅ Imports carregados com sucesso!")
print(f"📅 Data/Hora: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

## 📥 CÉLULA 2: Criar Estado Inicial (INPUT)

Este é o "input" do fluxo - simulando um webhook recebido da Evolution API.

In [ ]:
# Webhook simulado da Evolution API
webhook_data = {
    "body": {
        "event": "messages.upsert",
        "instance": "test-instance",
        "data": {
            "key": {
                "remoteJid": "5562999999999@s.whatsapp.net",
                "id": "MSG123456ABC",
                "fromMe": False
            },
            "pushName": "João Silva",
            "message": {
                "conversation": "Olá, quero saber sobre os serviços de drywall e gesso. Vocês fazem orçamento?"
            },
            "messageType": "conversation",
            "messageTimestamp": 1729522800
        }
    }
}

# Criar estado inicial
# state = criar_estado_inicial()
state = {
    "raw_webhook_data": webhook_data,
    "next_action": ""
}

print("="*60)
print("📥 INPUT - Estado Inicial")
print("="*60)
print(f"\n📱 Webhook recebido de: {webhook_data['body']['data']['pushName']}")
print(f"📞 Número: {webhook_data['body']['data']['key']['remoteJid']}")
print(f"💬 Mensagem: {webhook_data['body']['data']['message']['conversation']}")
print(f"📧 Tipo: {webhook_data['body']['data']['messageType']}")
print("\n" + "="*60)

## 🔍 CÉLULA 3: Testar NÓ 1 - validar_webhook

**Função:** Valida webhook e extrai dados

**Input:** `raw_webhook_data`  
**Output:** `cliente_numero`, `cliente_nome`, `mensagem_tipo`, `next_action`

In [ ]:
print("="*60)
print("🔍 TESTANDO NÓ 1: validar_webhook")
print("="*60)

# Executar o nó
# resultado = await validar_webhook(state)
# Para demonstração, simular resultado:
resultado = {
    **state,
    "cliente_numero": "5562999999999",
    "cliente_nome": "João Silva",
    "mensagem_tipo": "conversation",
    "mensagem_id": "MSG123456ABC",
    "mensagem_base64": "Olá, quero saber sobre os serviços de drywall e gesso. Vocês fazem orçamento?",
    "mensagem_from_me": False,
    "next_action": "verificar_cliente"
}

print("\n📤 OUTPUT do validar_webhook:")
print("-"*60)
print(f"✓ Cliente número: {resultado.get('cliente_numero')}")
print(f"✓ Cliente nome: {resultado.get('cliente_nome')}")
print(f"✓ Tipo mensagem: {resultado.get('mensagem_tipo')}")
print(f"✓ Mensagem ID: {resultado.get('mensagem_id')}")
print(f"✓ Próxima ação: {resultado.get('next_action')}")
print("="*60)

# Atualizar state para próximo nó
state = resultado

## 🔍 CÉLULA 4: Testar NÓ 2 - verificar_cliente

**Função:** Verifica se cliente existe no Supabase

**Input:** `cliente_numero`  
**Output:** `cliente_existe`, `cliente_id`, `next_action`

In [ ]:
print("="*60)
print("🔍 TESTANDO NÓ 2: verificar_cliente")
print("="*60)

print(f"\n📥 INPUT: Cliente número = {state.get('cliente_numero')}")

# Executar o nó (async)
# resultado = await verificar_cliente(state)
# Para demonstração, simular dois cenários:

# CENÁRIO A: Cliente NÃO encontrado
resultado = {
    **state,
    "cliente_existe": False,
    "cliente_id": None,
    "next_action": "cadastrar_cliente"
}

# # CENÁRIO B: Cliente encontrado (descomente para testar)
# resultado = {
#     **state,
#     "cliente_existe": True,
#     "cliente_id": "abc123xyz",
#     "next_action": "processar_midia"
# }

print("\n📤 OUTPUT do verificar_cliente:")
print("-"*60)
print(f"✓ Cliente existe: {resultado.get('cliente_existe')}")
print(f"✓ Cliente ID: {resultado.get('cliente_id')}")
print(f"✓ Próxima ação: {resultado.get('next_action')}")
print("="*60)

state = resultado

## 🔍 CÉLULA 5: Testar NÓ 3 - cadastrar_cliente (se necessário)

**Função:** Cadastra novo cliente no Supabase

**Input:** `cliente_nome`, `cliente_numero`, `mensagem_base64`, `mensagem_tipo`  
**Output:** `cliente_id`, `cliente_existe`, `next_action`

**Nota:** Execute esta célula apenas se `cliente_existe == False`

In [ ]:
if not state.get("cliente_existe"):
    print("="*60)
    print("🔍 TESTANDO NÓ 3: cadastrar_cliente")
    print("="*60)
    
    print(f"\n📥 INPUT: Cadastrando cliente {state.get('cliente_nome')}")
    
    # Executar o nó
    # resultado = await cadastrar_cliente(state)
    resultado = {
        **state,
        "cliente_id": "novo_cliente_xyz789",
        "cliente_existe": True,
        "next_action": "processar_midia"
    }
    
    print("\n📤 OUTPUT do cadastrar_cliente:")
    print("-"*60)
    print(f"✓ Cliente cadastrado com ID: {resultado.get('cliente_id')}")
    print(f"✓ Cliente existe: {resultado.get('cliente_existe')}")
    print(f"✓ Próxima ação: {resultado.get('next_action')}")
    print("="*60)
    
    state = resultado
else:
    print("⏭️  Cliente já existe, pulando cadastro")

## 🔍 CÉLULA 6: Testar NÓ 4 - processar_texto

**Função:** Processa mensagem de texto

**Input:** `mensagem_base64`  
**Output:** `mensagem_conteudo`, `mensagem_transcrita`, `next_action`

In [ ]:
print("="*60)
print("🔍 TESTANDO NÓ 4: processar_texto")
print("="*60)

print(f"\n📥 INPUT: {state.get('mensagem_base64')[:50]}...")

# Executar o nó
# resultado = processar_texto(state)
resultado = {
    **state,
    "mensagem_conteudo": state.get("mensagem_base64"),
    "mensagem_transcrita": state.get("mensagem_base64"),
    "next_action": "gerenciar_fila"
}

print("\n📤 OUTPUT do processar_texto:")
print("-"*60)
print(f"✓ Conteúdo: {resultado.get('mensagem_conteudo')[:50]}...")
print(f"✓ Próxima ação: {resultado.get('next_action')}")
print("="*60)

state = resultado

## 🔍 CÉLULA 7: Testar NÓ 5 - gerenciar_fila

**Função:** Adiciona mensagem à fila Redis

**Input:** `cliente_numero`, `mensagem_conteudo`  
**Output:** `fila_mensagens`, `deve_processar`, `next_action`

In [ ]:
print("="*60)
print("🔍 TESTANDO NÓ 5: gerenciar_fila")
print("="*60)

print(f"\n📥 INPUT: Cliente {state.get('cliente_numero')}")

# Executar o nó (async)
# resultado = await gerenciar_fila(state)
# Simular primeira mensagem (deve processar)
resultado = {
    **state,
    "fila_mensagens": [
        {
            "conteudo": state.get("mensagem_conteudo"),
            "timestamp": "2025-10-21T10:00:00",
            "tipo": state.get("mensagem_tipo")
        }
    ],
    "deve_processar": True,
    "next_action": "aguardar_mensagens"
}

print("\n📤 OUTPUT do gerenciar_fila:")
print("-"*60)
print(f"✓ Mensagens na fila: {len(resultado.get('fila_mensagens', []))}")
print(f"✓ Deve processar: {resultado.get('deve_processar')}")
print(f"✓ Próxima ação: {resultado.get('next_action')}")
print("="*60)

state = resultado

## 📊 CÉLULA 8: Visualizar Estado Completo

Estado atual após processar todos os nós.

In [ ]:
print("="*60)
print("📊 ESTADO COMPLETO ATUAL")
print("="*60)
print("\n🔍 Informações do Cliente:")
print(f"  - Número: {state.get('cliente_numero')}")
print(f"  - Nome: {state.get('cliente_nome')}")
print(f"  - Existe: {state.get('cliente_existe')}")
print(f"  - ID: {state.get('cliente_id')}")

print("\n💬 Informações da Mensagem:")
print(f"  - Tipo: {state.get('mensagem_tipo')}")
print(f"  - Conteúdo: {state.get('mensagem_conteudo', '')[:60]}...")
print(f"  - ID: {state.get('mensagem_id')}")

print("\n📋 Fila:")
print(f"  - Mensagens: {len(state.get('fila_mensagens', []))}")
print(f"  - Deve processar: {state.get('deve_processar')}")

print("\n➡️  Fluxo:")
print(f"  - Próxima ação: {state.get('next_action')}")

if state.get('erro'):
    print("\n❌ Erro:")
    print(f"  - Mensagem: {state.get('erro')}")

print("\n" + "="*60)
print("✅ Estado processado com sucesso!")
print("="*60)

## 🧹 CÉLULA 9: Reset - Criar Novo Teste

Execute esta célula para começar um novo teste do zero.

In [ ]:
print("🔄 Resetando estado...")

# Voltar para CÉLULA 2 para criar novo webhook
print("\n✅ Pronto para novo teste!")
print("➡️  Execute novamente a CÉLULA 2 com um novo webhook")

---

## 📚 Próximos Passos

Para continuar testando:

1. **Instalar dependências**: `pip install jupyter ipython`
2. **Iniciar Jupyter**: `jupyter notebook`
3. **Descomentar imports** na CÉLULA 1
4. **Executar célula por célula**

---

## 🎯 Fases Implementadas

- ✅ Fase 0: Preparação do Ambiente
- ✅ Fase 1: Modelo de Estado e Tipos
- ✅ Fase 2: Clientes Externos
- ✅ Fase 3: Webhook e Cadastro
- ⏳ Fase 4: Processamento de Mídia (em desenvolvimento)

---